In [ ]:
# default_exp softwares.fastq.minimap2

# minimap2

In [ ]:
# export

from pybiotools4p.softwares.base import Base, modify_cmd



In [ ]:
# export 

class Minimap2(Base):
    def __init__(self, software, fd):
        super(Minimap2, self).__init__(software)
        self._default = fd
        
    @modify_cmd
    def cmd_version(self):
        '''
        :return:
        '''
        return 'echo {repr} ;{software} --version'.format(
            repr=self.__repr__(),
            software=self._software
        )

    @modify_cmd
    def cmd_splice_align(self,reference,fastq,samtools,samtools_idx,outbam):
        
        return r'''
{software} {splice_align} -a {reference} {fastq} | {samtools_sam2bam} | {samtools_sort} 
{samtools_index}
        '''.format(
            software=self._software,
            splice_align=self._default['splice_align'],
            samtools_sam2bam=samtools.cmd_sam2bam(samtools_idx, '-', bamfile=None),
            samtools_sort=samtools.cmd_sort('-', sortbam=outbam),
            samtools_index=samtools.cmd_index(outbam),
            **locals()
        )

    @modify_cmd
    def cmd_nonsplice_align(self,reference,fastq,samtools,samtools_idx,outbam):
        
        return r'''
{software} {nonsplice_align} -a {reference} {fastq} | {samtools_sam2bam} | {samtools_sort} 
{samtools_index}
        '''.format(
            software=self._software,
            nonsplice_align=self._default['nonsplice_align'],
            samtools_sam2bam=samtools.cmd_sam2bam(samtools_idx, '-', bamfile=None),
            samtools_sort=samtools.cmd_sort('-', sortbam=outbam),
            samtools_index=samtools.cmd_index(outbam),
            **locals()
        )

    def __repr__(self):
        return 'minimap2:' + self._software

    def __str__(self):
        return 'A versatile pairwise aligner for genomic and spliced nucleotide sequences'

In [ ]:
import configparser

config=configparser.ConfigParser()
config.read('pybiotools4p/default.ini')

from pybiotools4p.softwares.bam.samtools import Samtools

samtools=Samtools('samtools',config['samtools'])
minimap2=Minimap2('minimap2',config['minimap2'])


In [ ]:
reference='biology-test-data/fasta/Homo_sapiens.GRCh38.dna.primary_assembly.chromosome22.fa'
samtools_idx='biology-test-data/fasta/Homo_sapiens.GRCh38.dna.primary_assembly.chromosome22.fa.fai'
fastq = '/path/to/fastq'
outbam='/path/to/bam'

In [ ]:
print(minimap2.cmd_splice_align(reference,fastq,samtools,samtools_idx,outbam))

minimap2 -ax splice -uf -k 14 -G 10000 -a biology-test-data/fasta/Homo_sapiens.GRCh38.dna.primary_assembly.chromosome22.fa /path/to/fastq | samtools view -bSt biology-test-data/fasta/Homo_sapiens.GRCh38.dna.primary_assembly.chromosome22.fa.fai - | samtools sort - -o /path/to/bam 
samtools index /path/to/bam


In [ ]:
print(minimap2.cmd_nonsplice_align(reference,fastq,samtools,samtools_idx,outbam))

minimap2 -ax map-ont -a biology-test-data/fasta/Homo_sapiens.GRCh38.dna.primary_assembly.chromosome22.fa /path/to/fastq | samtools view -bSt biology-test-data/fasta/Homo_sapiens.GRCh38.dna.primary_assembly.chromosome22.fa.fai - | samtools sort - -o /path/to/bam 
samtools index /path/to/bam
